# SQL query from table names - Continued

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("""give me name of the employee with the highest salary""", context_user))

```sql
SELECT name
FROM employees
ORDER BY salary DESC
LIMIT 1;
```


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("give me name of the employee with the highest salary", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee with the highest salary by joining the "employees" table with the "salary" table on the ID_usr column, then ordering the results by salary in descending order and limiting the output to only one row, which corresponds to the employee with the highest salary.


In [9]:
#new
print(return_CCRMSQL("give me the institution with the highest average salary?", context_user))

```sql
SELECT institution_name, AVG(salary) AS avg_salary
FROM employees
GROUP BY institution_name
ORDER BY avg_salary DESC
LIMIT 1;
```


In [10]:
#old
print(return_CCRMSQL("give me the institution with the highest average salary?", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(s.salary) AS avg_salary
FROM salary s
JOIN employees e ON s.ID_usr = e.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the `salary` table with the `employees` table on the `ID_usr` column. It calculates the average salary for each institution by grouping the results by `Institution`. Then, it orders the results in descending order based on the average salary and limits the output to only the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.

        They worked out nice without any problems.

     - What did you learn?

    I learned that defining clear SQL structures and ensuring valid syntax is crucial for accurate query generation. Missing commas or inconsistent field definitions can cause errors, and specifying database constraints helps maintain data integrity.
    

### Version 1

In [11]:
#new context
print(return_CCRMSQL("give me a list of all employees along with their salary and field of study.", context_user))

```sql
SELECT e.employee_name, e.salary, d.field_of_study
FROM employees e
JOIN departments d ON e.department_id = d.department_id;
```


In [13]:
#old context
print(return_CCRMSQL("give me a list of all employees along with their salary and field of study.", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary, st.educational_level, st.Institution, st.Speciality
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr;
```

This SQL query retrieves the name of employees, their salary, educational level, institution, and speciality by joining the employees table with the salary table on the employee ID and then joining the studies table on the employee ID.


### Version 2

In [14]:
#new context
print(return_CCRMSQL("give me how many employees studied at each institution?", context_user))

```sql
SELECT institution, COUNT(employee_id) AS num_employees
FROM employees
GROUP BY institution;
```


In [ ]:
#new context
print(return_CCRMSQL("give me how many employees studied at each institution?", context_user))

```sql
SELECT institution, COUNT(employee_id) AS num_employees
FROM employees
GROUP BY institution;
```


### Version 3

In [16]:
#new context
print(return_CCRMSQL("give me which employee had the highest salary in the year 2022?", context_user))

```sql
SELECT employee_name
FROM employees
WHERE salary = (SELECT MAX(salary) FROM employees WHERE year = 2022);
```


In [17]:
#old context
print(return_CCRMSQL("give me which employee had the highest salary in the year 2022?", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year = '2022'
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name and salary of the employee with the highest salary in the year 2022 by joining the "employees" and "salary" tables on the employee ID, filtering the results for the year 2022, ordering the salaries in descending order, and limiting the output to the top result.
